# Task 6: Basic data cleaning and consistency checks|

In [3]:
# Importing library

import pandas as pd
import os 

#creating path
path=r'C:\Users\Olufemi\Downloads\CareerFoundry Data Analyics\Exercise 6.1'

# Import dataframes
df_2015 = pd.read_csv(os.path.join(path, '6.1_csvs', '2015.csv'))
df_2016 = pd.read_csv(os.path.join(path, '6.1_csvs', '2016.csv'))
df_2017 = pd.read_csv(os.path.join(path, '6.1_csvs', '2017.csv'))
df_2018 = pd.read_csv(os.path.join(path, '6.1_csvs', '2018.csv'))
df_2019 = pd.read_csv(os.path.join(path, '6.1_csvs', '2019.csv'))

In [5]:
# Check structure and data types

print("2015 DataFrame Info")
df_2015.info()

print("\n2016 DataFrame Info")
df_2016.info()

print("\n2017 DataFrame Info")
df_2017.info()

print("\n2018 DataFrame Info")
df_2018.info()

print("\n2019 DataFrame Info")
df_2019.info()

2015 DataFrame Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Country                        158 non-null    object 
 1   Region                         158 non-null    object 
 2   Happiness Rank                 158 non-null    int64  
 3   Happiness Score                158 non-null    float64
 4   Standard Error                 158 non-null    float64
 5   Economy (GDP per Capita)       158 non-null    float64
 6   Family                         158 non-null    float64
 7   Health (Life Expectancy)       158 non-null    float64
 8   Freedom                        158 non-null    float64
 9   Trust (Government Corruption)  158 non-null    float64
 10  Generosity                     158 non-null    float64
 11  Dystopia Residual              158 non-null    float64
dtypes: float64(9), int64(1), objec

In [7]:
# Defines a dictionary dfs that maps each year (2015–2019) to its corresponding pandas DataFrame.

dfs = {
    "2015": df_2015,
    "2016": df_2016,
    "2017": df_2017,
    "2018": df_2018,
    "2019": df_2019
}

In [9]:
# Define Standard column mapping

standard_columns = {
    "Country": "Country",
    "Country or region": "Country",
    "Region": "Region",
    "Happiness Rank": "Rank",
    "Happiness.Rank": "Rank",
    "Overall rank": "Rank",
    "Happiness Score": "Score",
    "Happiness.Score": "Score",
    "Score": "Score",
    "Economy (GDP per Capita)": "GDP per capita",
    "Economy..GDP.per.Capita.": "GDP per capita",
    "GDP per capita": "GDP per capita",
    "Family": "Social support",
    "Social support": "Social support",
    "Health (Life Expectancy)": "Healthy life expectancy",
    "Health..Life.Expectancy.": "Healthy life expectancy",
    "Healthy life expectancy": "Healthy life expectancy",
    "Freedom": "Freedom",
    "Freedom to make life choices": "Freedom",
    "Trust (Government Corruption)": "Trust",
    "Trust..Government.Corruption.": "Trust",
    "Perceptions of corruption": "Trust",
    "Generosity": "Generosity",
    "Dystopia Residual": "Dystopia Residual"
}

In [11]:
# Define the final column list with Region and Income Group
final_columns = ["Country", "Region", "Income Group", "Year", "Rank", "Score", "GDP per capita",
                 "Social support", "Healthy life expectancy", "Freedom", "Trust", "Generosity", "Dystopia Residual"]

In [13]:
# Build region mapping from 2015 and 2016 into all DataFrames:
region_mapping = pd.concat([
    df_2015[["Country", "Region"]],
    df_2016[["Country", "Region"]]
]).drop_duplicates().set_index("Country")["Region"].to_dict()

In [15]:
# Load the World Bank income group classification file
income_df = pd.read_csv(os.path.join(path, '6.1_csvs', 'world_bank_income_groups.csv'))
income_mapping = income_df.set_index("Country Name")["IncomeGroup"].to_dict()

In [17]:
# Clean and process each dataset
cleaned_dfs = []

for year, df in dfs.items():
    # Rename columns to standard form
    df_renamed = df.rename(columns={col: standard_columns[col] for col in df.columns if col in standard_columns})
    
    # Add missing columns as NaN
    for col in final_columns:
        if col not in df_renamed.columns:
            df_renamed[col] = pd.NA

    # Add Year
    df_renamed["Year"] = int(year)

    # Fill in Region if missing
    if "Region" not in df.columns:
        df_renamed["Region"] = df_renamed["Country"].map(region_mapping)

    # Fill in Income Group
    df_renamed["Income Group"] = df_renamed["Country"].map(income_mapping)

    # Reorder columns
    df_final = df_renamed[final_columns]

    # Append to final list
    cleaned_dfs.append(df_final)

In [19]:
# Concatenate all years
merged_df = pd.concat(cleaned_dfs, ignore_index=True)

C:\Users\Olufemi\AppData\Local\Temp\ipykernel_22700\2358127122.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat(cleaned_dfs, ignore_index=True)


In [21]:
# Drop the Dystopia Residual column because it is totally irrelevant to the analysis

merged_df.drop(columns=["Dystopia Residual"], inplace=True)

In [23]:
# Save concatenated file to CSV
merged_df.to_csv(os.path.join(path, '6.1_csvs', 'Global_Happiness_2015_2019_Cleaned.csv'), index=False)

In [25]:
# Print output
print(merged_df.head())

       Country          Region Income Group  Year  Rank  Score  \
0  Switzerland  Western Europe  High income  2015     1  7.587   
1      Iceland  Western Europe  High income  2015     2  7.561   
2      Denmark  Western Europe  High income  2015     3  7.527   
3       Norway  Western Europe  High income  2015     4  7.522   
4       Canada   North America  High income  2015     5  7.427   

   GDP per capita  Social support  Healthy life expectancy  Freedom    Trust  \
0         1.39651         1.34951                  0.94143  0.66557  0.41978   
1         1.30232         1.40223                  0.94784  0.62877  0.14145   
2         1.32548         1.36058                  0.87464  0.64938  0.48357   
3         1.45900         1.33095                  0.88521  0.66973  0.36503   
4         1.32629         1.32261                  0.90563  0.63297  0.32957   

   Generosity  
0     0.29678  
1     0.43630  
2     0.34139  
3     0.34699  
4     0.45811  


In [27]:
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Country                  782 non-null    object 
 1   Region                   774 non-null    object 
 2   Income Group             677 non-null    object 
 3   Year                     782 non-null    int64  
 4   Rank                     782 non-null    int64  
 5   Score                    782 non-null    float64
 6   GDP per capita           782 non-null    float64
 7   Social support           782 non-null    float64
 8   Healthy life expectancy  782 non-null    float64
 9   Freedom                  782 non-null    float64
 10  Trust                    781 non-null    float64
 11  Generosity               782 non-null    float64
dtypes: float64(7), int64(2), object(3)
memory usage: 73.4+ KB
None


In [31]:
# Check for any exact duplicate rows
duplicates = merged_df[merged_df.duplicated()]
print(f"Number of duplicate rows: {duplicates.shape[0]}")

Number of duplicate rows: 0


In [33]:
# Descriptive statistical summary for numerical columns
print(merged_df.describe())

              Year        Rank       Score  GDP per capita  Social support  \
count   782.000000  782.000000  782.000000      782.000000      782.000000   
mean   2016.993606   78.698210    5.379018        0.916047        1.078392   
std       1.417364   45.182384    1.127456        0.407340        0.329548   
min    2015.000000    1.000000    2.693000        0.000000        0.000000   
25%    2016.000000   40.000000    4.509750        0.606500        0.869363   
50%    2017.000000   79.000000    5.322000        0.982205        1.124735   
75%    2018.000000  118.000000    6.189500        1.236187        1.327250   
max    2019.000000  158.000000    7.769000        2.096000        1.644000   

       Healthy life expectancy     Freedom       Trust  Generosity  
count               782.000000  782.000000  781.000000  782.000000  
mean                  0.612416    0.411091    0.125436    0.218576  
std                   0.248309    0.152880    0.105816    0.122321  
min                  